In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.4'
spark_version = 'spark-3.2.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
tar: spark-3.2.4-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)


In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [ ]:
# 3. What is the average price for a four bedroom house sold
#in each year rounded to two decimal places?

sql_query = """
SELECT
    YEAR(date_built) AS year,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    YEAR(date_built)
ORDER BY
    year
"""

result = spark.sql(sql_query)
result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2010|296800.75|
|2011| 302141.9|
|2012|298233.42|
|2013|299999.39|
|2014|299073.89|
|2015|307908.86|
|2016|296050.24|
|2017|296576.69|
+----+---------+



In [ ]:
# 4. What is the average price of a home for each year the home was
#built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

sql_query = """
SELECT
    YEAR(date_built) AS year,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3
    AND
    bathrooms = 3
GROUP BY
    YEAR(date_built)
ORDER BY
    year
"""

result = spark.sql(sql_query)
result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2010|292859.62|
|2011|291117.47|
|2012|293683.19|
|2013|295962.27|
|2014|290852.27|
|2015| 288770.3|
|2016|290555.07|
|2017|292676.79|
+----+---------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms,
#3 bathrooms, with two floors, and are greater than or equal to 2,000 square feet
#rounded to two decimal places?
sql_query = """
SELECT
    YEAR(date_built) AS year,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3
    AND
    bathrooms = 3
    AND
    floors = 2
    AND
    sqft_living >= 2000
GROUP BY
    YEAR(date_built)
ORDER BY
    year
"""

result = spark.sql(sql_query)
result.show()



+----+---------+
|year|avg_price|
+----+---------+
|2010|285010.22|
|2011|276553.81|
|2012|307539.97|
|2013|303676.79|
|2014|298264.72|
|2015|297609.97|
|2016| 293965.1|
|2017|280317.58|
+----+---------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places,
#where the homes are greater than or equal to $350,000?
#Although this is a small dataset, determine the run time for this query.

sql_query = """
SELECT
    ROUND(AVG(price), 2) AS avg_price,
    AVG(view) AS avg_view
FROM
    home_sales
WHERE
    price >= 350000
"""

result = spark.sql(sql_query)
# result.show()

start_time = time.time()


print("--- %s seconds ---" % (time.time() - start_time))

--- 9.965896606445312e-05 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Start measuring time
start_time = time.time()

# Run Spark SQL query on cached data
sql_query = """
SELECT
    ROUND(AVG(price), 2) AS avg_price,
    AVG(view) AS avg_view
FROM
    home_sales
WHERE
    price >= 350000
"""

result = spark.sql(sql_query)
# result.show()

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.05311298370361328 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').parquet('parquet_home_sales',mode='overwrite')

In [ ]:
# 11. Read the parquet formatted data.
parquet_homes_df = spark.read.parquet('parquet_home_sales')

In [ ]:
# 12. Create a temporary table for the parquet data.
df.createOrReplaceTempView("parquet_data")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

# Start measuring time
start_time = time.time()

# Run Spark SQL query on cached data
sql_query = """
SELECT
    ROUND(AVG(price), 2) AS avg_price,
    AVG(view) AS avg_view
FROM
    parquet_data
WHERE
    price >= 350000
"""

result = spark.sql(sql_query)
# result.show()

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.04087471961975098 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False